<a href="https://colab.research.google.com/github/Yashu2699/Deep_learning/blob/main/Semantic_seg/sandstone_dataset_multiclass_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb
import UNET_MultiClass
from UNET_MultiClass import multi_unet

In [ ]:
!pip install patchify

In [5]:
from keras.utils.np_utils import normalize
import os
import glob
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
from pathlib import Path

In [6]:
model = multi_unet(4, 128, 128, 1)
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 128, 128, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [7]:
model.load_weights('/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/h5_files/sandstone_multi_seg.hdf5')

In [ ]:
segm_images = []
path = "/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/sandstone_data/*.jpg"
from pathlib import Path
for file in glob.glob(path):  
    name = Path(file).stem 
    large_image = cv2.imread(file, 0)
    large_image = cv2.resize(large_image, (1024, 896))
    patches = patchify(large_image, (128, 128), step=128)  
    
    predicted_patches = []
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            #print(i,j)
            
            single_patch = patches[i,j,:,:]
            
            single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
            single_patch_input=np.expand_dims(single_patch_norm, 0)
    
            single_patch_prediction = (model.predict(single_patch_input))
            single_patch_predicted_img=np.argmax(single_patch_prediction, axis=3)[0,:,:]
    
            predicted_patches.append(single_patch_predicted_img)
    
    predicted_patches = np.array(predicted_patches)  
    predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 128,128) )  
    reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
    segm_images.append(reconstructed_image)
  

In [11]:
final_segm_image = np.array(segm_images).astype(np.uint8)   

from tifffile import imsave
imsave('/content/drive/MyDrive/ColabNotebooks/deep learning /Object_Segmentation/sandstone_segmented_test.tif', final_segm_image)

In [12]:
np.unique(final_segm_image)

array([0, 1, 2, 3], dtype=uint8)